In [ ]:
!pip install scikit-multilearn

time: 8.14 s (started: 2023-12-01 15:11:50 +00:00)


In [ ]:
!pip install ipython-autotime

time: 8.85 s (started: 2023-12-01 15:11:58 +00:00)


In [ ]:
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 4.53 ms (started: 2023-12-01 15:12:07 +00:00)


In [ ]:
import pandas as pd

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

import re

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score,recall_score,f1_score, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from skmultilearn.problem_transform import BinaryRelevance,LabelPowerset, ClassifierChain

import numpy as np

from keras.models import Sequential, load_model
from keras.preprocessing.text import Tokenizer
from keras.layers import LSTM, Embedding,Dropout, Dense
from keras.preprocessing.sequence import pad_sequences

time: 4.15 ms (started: 2023-12-01 16:29:33 +00:00)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def clean_text(text):
  # expanding short cuts to words, removing non word charecters
      text = text.lower()
      text = re.sub(r"what's", "what is ", text)
      text = re.sub(r"\'s", " ", text)
      text = re.sub(r"\'ve", " have ", text)
      text = re.sub(r"can't", "can not ", text)
      text = re.sub(r"n't", " not ", text)
      text = re.sub(r"i'm", "i am ", text)
      text = re.sub(r"\'re", " are ", text)
      text = re.sub(r"\'d", " would ", text)
      text = re.sub(r"\'ll", " will ", text)
      text = re.sub(r"\'scuse", " excuse ", text)
      text = re.sub('\W', ' ', text)
      text = re.sub('\s+', ' ', text)
      text = text.strip(' ')
      return text


# choosing an adequate value for smoothing factor by comparing accuracy
smoothing_list=[0.015]

TRAINING_DATA_FILE_PATH='./data/Training-dataset.csv'

VALIDATION_DATA_FILE_PATH='./data/Task-2-validation-dataset.csv'
VALIDATION_DATA_FILE_PATH_OUTPUT='./data/10664226-Task2-method-a-validation.csv'

TEST_DATA_FILE_PATH='./data/Task-2-test-dataset2.csv'
TEST_DATA_FILE_PATH_OUTPUT='./data/10664226-Task2-method-a.csv'

training_data=pd.read_csv(TRAINING_DATA_FILE_PATH)
validation_data=pd.read_csv(VALIDATION_DATA_FILE_PATH)
testing_data=pd.read_csv(TEST_DATA_FILE_PATH)








time: 1.84 s (started: 2023-12-01 15:12:07 +00:00)


In [ ]:
training_data['plot_synopsis']=training_data['title']+" "+training_data['plot_synopsis']


cleaned_training_data=training_data['plot_synopsis'].map(lambda com : clean_text(com))
cleaned_validation_data=validation_data['plot_synopsis'].map(lambda com : clean_text(com))
cleaned_testing_data=testing_data['plot_synopsis'].map(lambda com : clean_text(com))

stop_words = stopwords.words("english")

tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words)

time: 14.7 s (started: 2023-12-01 15:12:09 +00:00)


In [ ]:
# extracted vocabulary and calculated features for training data
transformed_training_data=tfidf_vectorizer.fit_transform(cleaned_training_data).toarray()

print("Tfidf count calculated....")

# these are the labels we need to train on
training_labels_matrix = training_data.drop(['ID', 'plot_synopsis','title'], axis=1)
training_labels=list(training_labels_matrix.columns.values)

Tfidf count calculated....
time: 21.8 s (started: 2023-12-01 15:12:24 +00:00)


In [ ]:
label_pwst_clf=None
for smoothing_value in smoothing_list:
  label_pwst_clf = LabelPowerset(MultinomialNB(alpha=smoothing_value,fit_prior=True))
  label_pwst_clf.fit(transformed_training_data, training_labels_matrix)
  print(f'Result calculated for {smoothing_value}')


Result calculated for 0.015
time: 26.6 s (started: 2023-12-01 15:12:46 +00:00)


In [ ]:
def predict_and_write(data_to_predict,FILE_PATH_OUTPUT):
  transformed_data_to_predict=tfidf_vectorizer.transform(data_to_predict)
  result_matrix=label_pwst_clf.predict(transformed_data_to_predict)

  df_out=pd.DataFrame(data=result_matrix.toarray(), columns=training_labels)

  df_out.insert(0, "ID", training_data['ID'] , True)

  df_out.to_csv(FILE_PATH_OUTPUT,index=False,header=None)
  return df_out,result_matrix


time: 885 µs (started: 2023-12-01 15:13:12 +00:00)


In [ ]:
predict_and_write(cleaned_validation_data,VALIDATION_DATA_FILE_PATH_OUTPUT)

(                                        ID  comedy  cult  flashback  \
 0     8f5203de-b2f8-4c0c-b0c1-835ba92422e9       0     0          0   
 1     6416fe15-6f8a-41d4-8a78-3e8f120781c7       0     0          0   
 2     4979fe9a-0518-41cc-b85f-f364c91053ca       0     0          0   
 3     b672850b-a1d9-44ed-9cff-025ee8b61e6f       0     0          1   
 4     b4d8e8cc-a53e-48f8-be6a-6432b928a56d       0     0          0   
 ...                                    ...     ...   ...        ...   
 1183  a46740d6-7a97-4b66-823b-3977a7a2fb8b       0     0          0   
 1184  6a1469cd-e168-4e1a-bd29-bad7c50b40d7       0     0          0   
 1185  7f675405-f185-4789-99f9-db8a7277616e       0     0          0   
 1186  ee2f4d92-967d-4043-a304-5d3d5caf24a9       0     0          0   
 1187  17662ec1-3c61-4292-99ec-0393424be25c       0     0          0   
 
       historical  murder  revenge  romantic  scifi  violence  
 0              0       0        0         1      0         0  
 1    

time: 3.67 s (started: 2023-12-01 15:13:12 +00:00)


In [ ]:
predict_and_write(cleaned_testing_data,TEST_DATA_FILE_PATH_OUTPUT)

(                                        ID  comedy  cult  flashback  \
 0     8f5203de-b2f8-4c0c-b0c1-835ba92422e9       0     0          0   
 1     6416fe15-6f8a-41d4-8a78-3e8f120781c7       0     0          0   
 2     4979fe9a-0518-41cc-b85f-f364c91053ca       1     0          0   
 3     b672850b-a1d9-44ed-9cff-025ee8b61e6f       0     0          0   
 4     b4d8e8cc-a53e-48f8-be6a-6432b928a56d       0     0          0   
 ...                                    ...     ...   ...        ...   
 1195  f9517a09-27cc-41f5-8d34-0a0af46a8697       0     0          0   
 1196  de4386f2-a4a2-4b55-a3a5-3026018935c4       0     0          0   
 1197  4f67a350-eb92-45c7-9e83-7cbc50473ca1       0     0          0   
 1198  18f4cdc5-8590-444e-93ce-45a83532f08b       0     0          0   
 1199  da5a0552-e852-4792-a74c-475703b617d1       1     0          0   
 
       historical  murder  revenge  romantic  scifi  violence  
 0              0       1        0         0      0         0  
 1    

time: 5.08 s (started: 2023-12-01 15:13:16 +00:00)


Start of Task 2 Method b : -

In [ ]:
TRAINING_DATA_FILE_PATH='./data/Training-dataset.csv'

VALIDATION_DATA_FILE_PATH='./data/Task-2-validation-dataset.csv'
VALIDATION_DATA_FILE_PATH_OUTPUT='./data/10664226-Task2-method-b-validation.csv'

TEST_DATA_FILE_PATH='./data/Task-2-test-dataset2.csv'
TEST_DATA_FILE_PATH_OUTPUT='./data/10664226-Task2-method-b.csv'

training_data=pd.read_csv(TRAINING_DATA_FILE_PATH)
validation_data=pd.read_csv(VALIDATION_DATA_FILE_PATH)
testing_data=pd.read_csv(TEST_DATA_FILE_PATH)

time: 938 ms (started: 2023-12-01 15:13:21 +00:00)


In [ ]:
# expanding short cuts to words, removing non word charecters, removing stop words

def clean_text_2(text,stop_words):
      text = text.lower()
      text = re.sub(r"what's", "what is ", text)
      text = re.sub(r"\'s", " ", text)
      text = re.sub(r"\'ve", " have ", text)
      text = re.sub(r"can't", "can not ", text)
      text = re.sub(r"n't", " not ", text)
      text = re.sub(r"i'm", "i am ", text)
      text = re.sub(r"\'re", " are ", text)
      text = re.sub(r"\'d", " would ", text)
      text = re.sub(r"\'ll", " will ", text)
      text = re.sub("\d+", " ", text)
      text = re.sub(r"\'scuse", " excuse ", text)
      text = re.sub('\W', ' ', text)
      text = re.sub('\s+', ' ', text)
      text = text.strip(' ')
      text=text.split()
      text=[w for w in text if not w in stop_words]
      return( " ".join(text))

def max_len(x):
    a=x.split()
    return len(a)

training_data['plot_synopsis']=training_data['title']+" "+training_data['plot_synopsis']


training_data=pd.read_csv(TRAINING_DATA_FILE_PATH)
development_data=pd.read_csv(VALIDATION_DATA_FILE_PATH)

stop_words = set(stopwords.words('english'))

vect=Tokenizer()

cleaned_training_data=training_data['plot_synopsis'].map(lambda com : clean_text_2(com,stop_words))

cleaned_training_list=cleaned_training_data.tolist()
cleaned_validation_list=cleaned_validation_data.tolist()

# this gets the maximum tokens found in a plot
max_length=max(cleaned_training_data.apply(max_len))
# tokenized
vect.fit_on_texts(cleaned_training_data)
vocab_size = len(vect.word_index) + 1

# converting tokenized data to numerical form
encoded_docs_train= vect.texts_to_sequences(cleaned_training_list)

# maxlen was chosen after testing for several maxlen values to compare performance (precision, recall, f1 score)
padded_docs_train = pad_sequences(encoded_docs_train, maxlen=int(0.75 * max_length), padding='post')




<class 'numpy.ndarray'>
time: 31.4 s (started: 2023-12-01 15:13:49 +00:00)


In [ ]:
model= Sequential()

model.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=int(0.75 * max_length)))

model.add(LSTM(64, return_sequences=True))

model.add(Dropout(0.5))

model.add(LSTM(32))

model.add(Dropout(0.5))

training_labels_matrix = training_data.drop(['ID', 'plot_synopsis','title'], axis=1)
training_labels_matrix_np=training_labels_matrix.to_numpy()
training_labels=list(training_labels_matrix.columns.values)

model.add(Dense(len(training_labels), activation='sigmoid'))

model.summary()

model.compile(optimizer='adam', loss='binary_crossentropy')



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2019, 100)         8886800   
                                                                 
 lstm (LSTM)                 (None, 2019, 64)          42240     
                                                                 
 dropout (Dropout)           (None, 2019, 64)          0         
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense (Dense)               (None, 9)                 297       
                                                                 
Total params: 8941753 (34.11 MB)
Trainable params: 89417

In [ ]:
history = model.fit(padded_docs_train, training_labels_matrix_np,
                    class_weight=None,
                    epochs=5,
                    batch_size=32,
                    validation_split=0.1,
                    callbacks=[])

model.save("./data/lstm_result.keras")

In [ ]:
model = load_model('./data/lstm_result.keras')


time: 5.4 s (started: 2023-12-01 16:34:08 +00:00)


In [ ]:

def cleaned_predict_and_write(data_to_predict,FILE_PATH_OUTPUT):

  cleaned_data_to_predict=data_to_predict['plot_synopsis'].map(lambda com : clean_text_2(com,stop_words))

  encoded_docs_test= vect.texts_to_sequences(cleaned_data_to_predict)

  padded_docs_test = pad_sequences(encoded_docs_test, maxlen=int(0.75 * max_length), padding='post')

  predictions=model.predict([padded_docs_test])

  # threshold for detection of positive value for class was set on the basis of performance
  thresholds=[0.3]
  for val in thresholds:
      pred=predictions.copy()

      predictions[pred>=val]=1
      predictions[pred<val]=0
      predictions=predictions.astype(np.int32)

  df_out=pd.DataFrame(data=predictions, columns=training_labels)

  df_out.insert(0, "ID", data_to_predict['ID'] , True)

  df_out.to_csv(FILE_PATH_OUTPUT,index=False,header=None)
  return df_out




time: 944 µs (started: 2023-12-01 16:35:33 +00:00)


In [ ]:
cleaned_predict_and_write(validation_data,VALIDATION_DATA_FILE_PATH_OUTPUT)

38/38 [==============================] - 21s 539ms/step


ID  comedy  cult  flashback  \
0     cf32cb00-172d-40f2-a3c1-936e8a0d89d7       0     0          0   
1     df7e125e-2d59-40e4-a126-9397e3a0ef21       0     0          0   
2     49bc73f3-9179-41cd-9774-905c7a3ac91b       0     0          0   
3     0ed4822b-87af-44bc-a677-7f7abfdaccf3       0     0          0   
4     0b1b0fa4-43bc-41ba-9598-b3401894b96d       0     0          0   
...                                    ...     ...   ...        ...   
1183  d32be875-41c7-4e84-ac04-e1d3bc3df0fe       0     0          0   
1184  84e025dd-4b4e-403c-a3dd-34818b210857       0     0          0   
1185  3d291d3b-c0b5-47cc-8dc8-127dc93162e3       0     0          0   
1186  6c9b3034-56b3-42f6-874e-a821c9fd1a89       0     0          0   
1187  fbd1d334-e979-465c-9fb0-e173d2642630       0     0          0   

      historical  murder  revenge  romantic  scifi  violence  
0              0       1        0         0      0         1  
1              0       1        0         0      0         1  
2              0       1        0         0      0         1  
3              0       1        0         0      0         1  
4              0       1        0         0      0         1  
...          ...     ...      ...       ...    ...       ...  
1183           0       1        0         0      0         1  
1184           0       1        0         0      0         1  
1185           0       1        0         0      0         1  
1186           0       1        0         0      0         1  
1187           0       1        0         0      0         1  

[1188 rows x 10 columns]

time: 42.9 s (started: 2023-12-01 16:35:37 +00:00)


In [ ]:
cleaned_predict_and_write(testing_data,TEST_DATA_FILE_PATH_OUTPUT)

38/38 [==============================] - 19s 497ms/step


ID  comedy  cult  flashback  \
0     82c09d29-36c7-4ceb-b8d9-a0f9e8d2ba35       0     0          0   
1     210c183c-4b41-4aab-a3c0-74c0345ef80f       0     0          0   
2     12f4bdf4-1645-42e4-8cd6-1ed4501cdce2       0     0          0   
3     b28d8b62-906b-4414-b50e-4275020df712       0     0          0   
4     31c32f7e-324a-4883-8f0a-006ebc5b40be       0     0          0   
...                                    ...     ...   ...        ...   
1195  30aadf1e-9380-485a-af26-073d71c5636a       0     0          0   
1196  46eb3e30-1a85-4479-988e-837de506245c       0     0          0   
1197  ddb44c89-8fc6-4475-8fdb-accae7df5464       0     0          0   
1198  872a6ed1-fd69-4c5d-a815-f1c9b37342e1       0     0          0   
1199  85c26f5f-a0b5-4e99-a90f-77fc779c53cb       0     0          0   

      historical  murder  revenge  romantic  scifi  violence  
0              0       1        0         0      0         1  
1              0       1        0         0      0         1  
2              0       1        0         0      0         1  
3              0       1        0         0      0         1  
4              0       1        0         0      0         1  
...          ...     ...      ...       ...    ...       ...  
1195           0       1        0         0      0         1  
1196           0       1        0         0      0         1  
1197           0       1        0         0      0         1  
1198           0       1        0         0      0         1  
1199           0       1        0         0      0         1  

[1200 rows x 10 columns]

time: 23.5 s (started: 2023-12-01 16:33:04 +00:00)
